In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from prnn.utils.predictiveNet import PredictiveNet
from prnn.utils.agent import RandomActionAgent
import itertools
import torch
import random
from prnn.utils.env import make_env
from prnn.utils.general import saveFig
from prnn.utils.figures import TrainingFigure
import matplotlib.pyplot as plt
from prnn.analysis.SpatialTuningAnalysis import SpatialTuningAnalysis
from prnn.analysis.OfflineTrajectoryAnalysis import OfflineTrajectoryAnalysis
from prnn.analysis.representationalGeometryAnalysis import representationalGeometryAnalysis



In [ ]:
savefolder = 'BasicAnalysisFigs'

In [ ]:
#Example Net
netname = 'thRNN_5win'
netfolder = '/maskedk_panel/'
exseed = 102
predictiveNet = PredictiveNet.loadNet(netfolder+netname+'-SpeedHD-s'+str(exseed))

In [ ]:
env = predictiveNet.EnvLibrary[0]
agentname = 'RandomActionAgent'
action_probability = np.array([0.15,0.15,0.6,0.1,0,0,0])
agent = RandomActionAgent(env.action_space,action_probability)
place_fields, SI, decoder = predictiveNet.calculateSpatialRepresentation(env,agent,
                                             trainDecoder=True, trainHDDecoder = True)

In [ ]:
predictiveNet.calculateDecodingPerformance(env,agent,decoder,
                                            savename=netname, savefolder=savefolder,
                                          trajectoryWindow=5,
                                          timesteps=1000)

In [ ]:
STA = SpatialTuningAnalysis(predictiveNet,inputControl=True, untrainedControl=True)


In [ ]:
STA.TCExamplesFigure(netname,savefolder)

In [ ]:
sleepnoise = 0.03
isomap_neighbors = 15
RGA = representationalGeometryAnalysis(predictiveNet, noisestd=sleepnoise,
                                       withIsomap=True, n_neighbors = isomap_neighbors)

In [ ]:
RGA.WakeSleepFigure(netname,savefolder)

In [ ]:
predictiveNet.pRNN.rnn.cell

In [ ]:

b_adapt = 1
tau_adapt=100
OTA_adapt = OfflineTrajectoryAnalysis(predictiveNet, noisestd=sleepnoise,
                                   withIsomap=False, decoder=decoder, 
                                      withAdapt=True, b_adapt = b_adapt, tau_adapt=tau_adapt,
                                      calculateViewSimilarity=True,
                                       compareWake=True)

In [ ]:
OTA_adapt.SpontTrajectoryFigure('adaptation',savefolder, trajRange=(150,250))


In [ ]:
OTA_query = OfflineTrajectoryAnalysis(predictiveNet, noisemag = 0, noisestd=sleepnoise,
                               withIsomap=False, decoder=decoder,
                                     actionAgent=True, calculateViewSimilarity=True,
                               compareWake=True)

In [ ]:
OTA_query.SpontTrajectoryFigure('actionquery',savefolder, trajRange=(110,150))
